In [102]:
import pandas as pd
import numpy as np
import tensorflow as tf
import glob
import os
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import random
import cv2
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import roc_curve,auc
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.metrics import AUC
from tensorflow.keras.utils import plot_model
from tensorflow.keras.regularizers import l2

In [103]:
main_path = 'data'
sub_dirs = ['train', 'test', 'val']

def create_dataframe(main_path, sub_dir):
    data = {"file_path": [], "label": []}
    for label_dir, label in zip(['real', 'fake'], [1, 0]):
        folder_path = os.path.join(main_path, sub_dir, label_dir)
        for img_file in os.listdir(folder_path):
            data["file_path"].append(os.path.join(folder_path, img_file))
            data["label"].append(label)
    return pd.DataFrame(data)

for sub_dir in sub_dirs:
    df = create_dataframe(main_path, sub_dir)
    csv_path = f"{sub_dir}.csv" 
    df.to_csv(csv_path, index=False)
    print(f"Saved {csv_path} with {len(df)} entries.")

Saved train.csv with 11448 entries.
Saved test.csv with 3103 entries.
Saved val.csv with 1257 entries.


In [104]:
train_df = pd.read_csv("train.csv").sample(frac=1, random_state=42).reset_index(drop=True)
valid_df = pd.read_csv("val.csv").sample(frac=1, random_state=42).reset_index(drop=True)
test_df = pd.read_csv("test.csv").sample(frac=1, random_state=42).reset_index(drop=True)

In [105]:
train_df['label'] = train_df['label'].astype(str)
valid_df['label'] = valid_df['label'].astype(str)
test_df['label'] = test_df['label'].astype(str)

In [106]:
# Define image size and batch size
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 64

# Preprocessing (rescale only)
train_datagen = ImageDataGenerator(rescale=1.0/255,horizontal_flip = True)
datagen = ImageDataGenerator(rescale=1.0/255)

# Training generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='file_path',
    y_col='label',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

# Validation generator
valid_generator = datagen.flow_from_dataframe(
    dataframe=valid_df,
    x_col='file_path',
    y_col='label',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# Test generator
test_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='file_path',
    y_col='label',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 11448 validated image filenames belonging to 2 classes.
Found 1257 validated image filenames belonging to 2 classes.
Found 3103 validated image filenames belonging to 2 classes.


In [107]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    # Load Xception without the top layer
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(256, 256, 3))
    # Add custom layers on top of the base model
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(512, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(1, activation='sigmoid')(x)

    # Create the full model
    model = Model(inputs=base_model.input, outputs=x)

    # Compile the model
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy", AUC(name='auc')])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [108]:
checkpoint_path = 'ff++_resnet50_model.keras'

model_checkpoint = ModelCheckpoint(
    checkpoint_path,
    monitor="val_accuracy",
    save_best_only=True,
    mode="max",
    verbose=1
)

history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=20,
    callbacks=[model_checkpoint]
)

model.load_weights(checkpoint_path)

/home/haipham2407/miniconda3/envs/qvit/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

2025-06-03 11:31:17.433185: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 256.00MiB (rounded to 268435456)requested by op StatefulPartitionedCall/functional_12_1/conv1_conv_1/convolution
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-06-03 11:31:17.433271: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1058] BFCAllocator dump for GPU_0_bfc
2025-06-03 11:31:17.433280: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (256): 	Total Chunks: 568, Chunks in use: 566. 142.0KiB allocated for chunks. 141.5KiB in use in bin. 31.9KiB client-requested in use in bin.
2025-06-03 11:31:17.433284: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (512): 	Total Chunks: 138, Chunks in use: 138. 69.5KiB allocated for chunks. 69.5

ResourceExhaustedError: Graph execution error:

Detected at node functional_12_1/conv1_conv_1/convolution defined at (most recent call last):
  File "/home/haipham2407/miniconda3/envs/qvit/lib/python3.11/threading.py", line 1002, in _bootstrap

  File "/home/haipham2407/miniconda3/envs/qvit/lib/python3.11/threading.py", line 1045, in _bootstrap_inner

  File "/home/haipham2407/miniconda3/envs/qvit/lib/python3.11/site-packages/keras/src/backend/tensorflow/trainer.py", line 58, in train_step

  File "/home/haipham2407/miniconda3/envs/qvit/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 117, in error_handler

  File "/home/haipham2407/miniconda3/envs/qvit/lib/python3.11/site-packages/keras/src/layers/layer.py", line 936, in __call__

  File "/home/haipham2407/miniconda3/envs/qvit/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 117, in error_handler

  File "/home/haipham2407/miniconda3/envs/qvit/lib/python3.11/site-packages/keras/src/ops/opera